# Bread Board Constrcut record.py

In [ ]:
import os

import numpy as np
import pandas as pd

from pyaspect.moment_tensor import MomentTensor

## paths

In [ ]:
data_in_dir  = 'data/output/'
data_out_dir = data_in_dir
!ls {data_out_dir}/tmp/TestProjects/CGFR_Test

In [ ]:
projects_fqp = os.path.join(data_out_dir,'tmp','TestProjects','CGFR_Test')
recip_project_fqp = os.path.join(projects_fqp,'ReciprocalGeometricTestProject')
fwd_project_fqp = os.path.join(projects_fqp,'ForwardGeometricTestProject')
!ls {recip_project_fqp}
print()
!ls {fwd_project_fqp}

## Record Object

In [ ]:
import os
import copy
import importlib

import numpy as np
import pandas as pd

from pyaspect.specfemio.headers import RecordHeader

#TODO this is the actual record. I need the header, then make record.py
class Record(RecordHeader):

            

    class _TraceData(object):
        
        class _XYZ(object):

            def __init__(self,ex,ny,z):

                self.ex = ex
                self.ny = ny
                self.z  = z

            def __str__(self):
                out_str  = f'Component E/X:\n{self.ex}\n\n'
                out_str += f'Component N/Y:\n{self.ny}\n\n'
                out_str += f'Component Z:\n{self.z}'
                return out_str

            def __repr__(self):
                out_str  = f'Component E/X:\n{self.ex.__repr__()}\n\n'
                out_str += f'Component N/Y:\n{self.ny.__repr__()}\n\n'
                out_str += f'Component Z:\n{self.z.__repr__()}'
                return out_str


        def __init__(self,data_df):
            
            self.df_x = data_df['comp_EX']
            self.df_y = data_df['comp_NY']
            self.df_z = data_df['comp_Z']


        def __getitem__(self,islice):
            return self._XYZ(self.df_x.loc[islice],self.df_y.loc[islice],self.df_z.loc[islice])

        def __str__(self):
            out_str  = f'Component E/X:\n{self.df_x}\n\n'
            out_str += f'Component N/Y:\n{self.df_y}\n\n'
            out_str += f'Component Z:\n{self.df_z}'
            return out_str

        def __repr__(self):
            out_str  = f'Component E/X:\n{self.df_x.__repr__()}\n\n'
            out_str += f'Component N/Y:\n{self.df_y.__repr__()}\n\n'
            out_str += f'Component Z:\n{self.df_z.__repr__()}'
            return out_str
        
            

    def __init__(self, rheader,dtype='b',data_df=None):
        super(Record,self).__init__(name=rheader.name,
                                    solutions_h=rheader.get_solutions_header_list(),
                                    stations_h=rheader.get_stations_header_list(),
                                    proj_id=rheader.proj_id,
                                    rid=rheader.rid,
                                    iter_id=rheader.iter_id,
                                    is_reciprocal=rheader.is_reciprocal)
        
        if not isinstance(data_df,pd.DataFrame):
            self._load_data(dtype=dtype)
        else:
            self['data_df'] = data_df
            

    def __str__(self):
        out_str  = f'{super(Record, self).__str__()}\n\n'
        out_str += f'Data:\n {self.data_df}'
        return out_str
    
    def __repr__(self):
        out_str  = f'{super(Record, self).__repr__()}\n\n'
        out_str += f'Data:\n {self.data_df.__repr__()}'
        return out_str
    
    def __getitem__(self, kslice):
        
        if not isinstance(kslice, str):
            dslice = super(Record, self)._get_df_slice_index(kslice,self.data_df,is_stations=True)
            c_data_df = self.data_df.reset_index()[dslice]
            c_rheader = super(Record, self).__getitem__(kslice)
            return Record(rheader=c_rheader,data_df=c_data_df)
        else:
            return super(Record, self).__getitem__(kslice)
    
    def _read_specfem_bin_trace(self,fpath,dtype=np.float32):
        return np.fromfile(fpath, dtype=dtype)

    def _load_data(self,dtype='b',sl=slice(None,None,None),scale=1.0,rfunc=None):

        if dtype != 'b' and _rfunc == None:
            raise Exception('can only read binary type data for the time being')
            
        #FIXME: add read ascii
        read_func = self._read_specfem_bin_trace
        if rfunc != None:
            read_func = rfunc

        l_data = []
        for eidx, edf in self.stations_df.groupby(level='eid'):
            for sidx, sdf in edf.groupby(level='sid'):
                for tidx, tdf in sdf.groupby(level='trid'):
                    for gidx, gdf in tdf.groupby(level='gid'):
                        fp_prefix = gdf.loc[(eidx,sidx,tidx,gidx),"data_fqdn"]
                        fp = os.path.join(projects_fqp,fp_prefix)
                        match_fp = fp + '.*X[XYZEN].sem*'
                        data_dict = {'eid':eidx,'sid':sidx,'trid':tidx,'gid':gidx}
                        for filepath in glob.glob(match_fp):
                            comp = filepath.split('.')[-2][-1]
                            if comp == 'X' or comp == 'E':
                                data_dict['comp_EX'] = scale*read_func(filepath)
                            elif comp == 'Y' or comp == 'N':
                                data_dict['comp_NY'] = scale*read_func(filepath)
                            elif comp == 'Z':
                                data_dict['comp_Z'] = scale*read_func(filepath)
                            else:
                                raise Exception(f'Could not find component: "{comp}"')
                                
                        l_data.append(data_dict)
                            
        self['data_df'] = pd.DataFrame.from_records(l_data, index=self['default_stat_midx'])
        
        
    @property
    def data(self):
        return self._TraceData(self.data_df)
        
    @property
    def data_df(self):
        return self['data_df']
    
    @property
    def component_names(self):
        return ['comp_EX','comp_NY','comp_Z']
    

## Reciprocity: Read RecordHeader and instantiate RecordObject

In [ ]:
import glob
from pyaspect.specfemio.read import _read_headers

recip_record_fqp = os.path.join(recip_project_fqp,'pyheader.project_record')
recip_record_h = _read_headers(recip_record_fqp)

recip_record_h.is_reciprocal = True #just a hack until updated

ne = recip_record_h.nevents
ns = recip_record_h.nsrc

print(f'ne:{ne}, ns:{ns}')
print(f'Recip Header:\n{recip_record_h.solutions_df.loc[pd.IndexSlice[:,1],:]}')

## Instantiate Record, and test slicing and pandas operations with DataFrames

In [ ]:
recip_record = Record(recip_record_h)
#print(recip_drecord['is_reciprocal'])
#print(recip_drecord.data_df.loc[(0,0,0),:])
#print(recip_drecord.data_df.loc[:,'comp_EX'])
#print(recip_drecord[0,0,0,:])
data = recip_record.data
#print(type(data[0,0,0,0].z))
#print(data)
print(pd.merge(recip_record.stations_df,recip_record.data_df,on=['eid','sid','trid','gid']))


## trace spacial derivative function to add to records.py module

In [ ]:
from scipy import signal

def calulate_spacial_derivative(tdf,eidx,sidx,tidx,g_p1,g_m1,sos,comp_key,coord_key):
    gidx_0  = pd.IndexSlice[eidx,sidx,tidx,0]
    gidx_p1 = pd.IndexSlice[eidx,sidx,tidx,g_p1]
    gidx_m1 = pd.IndexSlice[eidx,sidx,tidx,g_m1]
    df_0    = tdf.loc[gidx_0]
    df_p1   = tdf.loc[gidx_p1]
    df_m1   = tdf.loc[gidx_m1]
    data_p1 = signal.sosfilt(sos, df_p1[comp_key].astype(np.float64))
    data_m1 = signal.sosfilt(sos, df_m1[comp_key].astype(np.float64))
    c_p1    = df_p1[coord_key]
    c_m1    = df_m1[coord_key]
    c_0     = df_0[coord_key]
    delta   = 0.5*(c_p1 - c_m1)
    h       = 2.0*np.abs(delta)
    c       = c_m1 + delta
    
    assert h != 0
    assert c_0-c == 0
    
    h_scale  = 1/h
    mt_trace = h_scale*(data_p1 - data_m1)
    
    return mt_trace

## make reciprocal Green's functions: add to record.py module

In [ ]:
import pandas as pd
from scipy import signal

def make_rgf_data_df(record,fl,fh,fs):

    comp_dict = {'comp_EX':0,'comp_NY':1,'comp_Z':2}
    coord_dict = {0:'lon_xc',1:'lat_yc',2:'depth'}
    sos = signal.butter(3, [fl,fh], 'bp', fs=fs, output='sos') 

    l_rgf_traces = []
    m_df = pd.merge(record.stations_df,record.data_df,on=['eid','sid','trid','gid'])
    for eidx, edf in m_df.groupby(level='eid'):
        for sidx, sdf in edf.groupby(level='sid'):
            for tidx, tdf in sdf.groupby(level='trid'):
                for comp_key in comp_dict.keys():
                    ie = tidx
                    ig = eidx
                    fi = sidx
                    for di in range(3):
                        rgf_dict = {'eid':tidx,'trid':eidx,'fid':sidx}
                        rgf_dict['cid'] = comp_dict[comp_key]
                        coord_key = coord_dict[di]
                        rgf_dict['did'] = di
                        ip1 = di+1     #coord + h
                        im1 = ip1 + 3  #coord - h
                        if di == 2:
                            tm1 = ip1
                            ip1 = im1
                            im1 = tm1
                        rgf_dict['data'] = calulate_spacial_derivative(m_df,
                                                                       eidx,
                                                                       sidx,
                                                                       tidx,
                                                                       ip1,
                                                                       im1,
                                                                       sos,
                                                                       comp_key,
                                                                       coord_key)
                        
                        l_rgf_traces.append(rgf_dict)
                    
    return pd.DataFrame.from_records(l_rgf_traces, index=('eid','trid','cid','fid','did'))


## create Reciprocal Green's Table (as DataFrame)

In [ ]:
rgf_data_df = make_rgf_data_df(recip_record,1.0,10.0,1000)
rgf_data_df

In [ ]:
#print(rgf_df)
print(rgf_data_df.loc[0,0,0,:,:])

## Forward/CMTSolution: Read RecordHeader and instantiate RecordObject

In [ ]:
fwd_record_fqp = os.path.join(fwd_project_fqp,'pyheader.project_record')
fwd_record_h = _read_headers(fwd_record_fqp)

fwd_record_h.is_reciprocal = False #just a hack until updated

ne = fwd_record_h.nevents
ns = fwd_record_h.nsrc

print(f'ne:{ne}, ns:{ns}')
print(f'Forward Record:\n{fwd_record_h.solutions_df.loc[(0,0),"date"]}')

## Instantiate Forward Record

In [ ]:
fwd_record = Record(fwd_record_h)
#print(fwd_record['is_reciprocal'])
#print(fwd_record.data_df.loc[(0,0,0),:])
#print(fwd_record.data_df.loc[:,'comp_EX'])
#print(fwd_record[0,0,0,:])
data = fwd_record.data
#print(type(data[0,0,0,0].z))
print(data)

## Get Moment tensors to compare with Foward data and also Construct Combinded Reciprocal CMTs. These functions will not be part of record.py module, but make_moment_tensor will be added to utils.py module

In [ ]:
def make_moment_tensor(src_h):
    
    mrr = src_h['mrr']
    mtt = src_h['mtt']
    mpp = src_h['mpp']
    mrt = src_h['mrt']
    mrp = src_h['mrp']
    mtp = src_h['mtp']
    
    h_matrix = np.array([[mrr,mrt,mrp],[mrt,mtt,mtp],[mrp,mtp,mpp]])
    
    return MomentTensor(m_up_south_east=h_matrix)


#print(f'Forward Record Sources:\n{fwd_record_h.solutions_df}')
SrcHeader = fwd_record_h.solution_cls

d_fwd_src = {}
for eidx, edf in fwd_record_h.solutions_df.groupby(level='eid'):
    for sidx, sdf in edf.groupby(level='sid'):
        idx = pd.IndexSlice[eidx,sidx]
        src = SrcHeader.from_series(fwd_record_h.solutions_df.loc[idx])
        #print(src)
        #mag    = src.mw
        #strike = src.strike
        #dip    = src.dip
        #rake   = src.rake
        #mt = MomentTensor(mw=mag,strike=strike,dip=dip,rake=rake)
        mt = make_moment_tensor(src)
        print(mt)
        d_fwd_src[eidx] = mt
        #print(f'mt.aki_m6:\n{mt.aki_richards_m6()}')
        #print(f'header.m6:\n{src.mt}\n')

for key in d_fwd_src:
    print(d_fwd_src[key].m6_up_south_east())

## Make Reciprocal CMT record from MomentTensors Function

In [ ]:
def make_cmt_data_df_from_rgf(rgf_df,mt_dict):
    
    comp_dict = {'comp_EX':0,'comp_NY':1,'comp_Z':2}
    
    rgf_events = list(rgf_df.index.get_level_values('eid').unique())
    #print(f'rgf_events: {rgf_events}')
    mt_events  = list(mt_dict.keys())
    #print(f'mt_events: {mt_events}')
    #print(f'all: {rgf_events == mt_events}')
    
    if not rgf_events == mt_events:
        raise Exception('RGF-events do not match MomentTensors-events')
    
    l_recip_cmt_traces = []
    for eidx, edf in rgf_df.groupby(level='eid'):
        
        mw = d_fwd_src[eidx].magnitude
        m0 = d_fwd_src[eidx].moment
        mt_arr = d_fwd_src[eidx].m6_up_south_east()
        
        wzz =  mt_arr[0] #mrr
        wyy =  mt_arr[1] #mtt
        wxx =  mt_arr[2] #mpp
        wyz = -mt_arr[3] #mrt
        wxz =  mt_arr[4] #mrp
        wxy = -mt_arr[5] #mtp
        
        #print(f'Mw:{mw:.2f}, M0:{m0:.2f}, wzz:{wzz:.3f}, wyy:{wyy:.3f}, wee:{wxx:.3f}, wxy:{wxy:.3f}, wxz:{wxz:.3f}, wyz:{wyz:.3f}')
    
        
        for tidx, tdf in edf.groupby(level='trid'):
            d_recip_cmt = {'eid':eidx,'sid':eidx,'trid':tidx,'gid':0}
            for comp_key in comp_dict.keys():
                ic = comp_dict[comp_key]
                
                composite_trace  = wxx*1*rgf_df.loc[(eidx,tidx, 0,ic, 0),'data'] #Matrix: Mee
                composite_trace += wyy*1*rgf_df.loc[(eidx,tidx, 1,ic, 1),'data'] #Matrix: Mnn
                composite_trace += wzz*1*rgf_df.loc[(eidx,tidx, 2,ic, 2),'data'] #Matrix: Mzz

                #Matrix: M1/Mxy
                composite_trace += wxy*1*rgf_df.loc[(eidx,tidx, 1,ic, 0),'data']
                composite_trace += wxy*1*rgf_df.loc[(eidx,tidx, 0,ic, 1),'data']

                #Matrix: M2/Mxz
                composite_trace += wxz*1*rgf_df.loc[(eidx,tidx, 0,ic, 2),'data']
                composite_trace += wxz*1*rgf_df.loc[(eidx,tidx, 2,ic, 0),'data']

                #Matrix: M3/Myz
                composite_trace += wyz*1*rgf_df.loc[(eidx,tidx, 1,ic, 2),'data']
                composite_trace += wyz*1*rgf_df.loc[(eidx,tidx, 2,ic, 1),'data']
                
                d_recip_cmt[comp_key] = composite_trace
                
            l_recip_cmt_traces.append(d_recip_cmt)
        
    return pd.DataFrame.from_records(l_recip_cmt_traces, index=('eid','sid','trid','gid'))
                

## Construct the Dataframe with the Reciprocal CMT Traces 

In [ ]:
rgf_cmt_data_df = make_cmt_data_df_from_rgf(rgf_data_df,d_fwd_src)
for eidx, edf in rgf_cmt_data_df.groupby(level='eid'):
    print(eidx)

print(rgf_cmt_data_df.loc[pd.IndexSlice[0,0,:,0],:])
assert False

## Construct the Reciprocal CMT RecordHeader and then a Record

In [ ]:
import datetime

from pyaspect.specfemio.headers import CMTSolutionHeader as cmt_h
from pyaspect.specfemio.headers import StationHeader     as stat_h
'''
hstr  = f'PDE {date.year} {date.month} {date.day} {date.hour} {date.minute} {date.second}'
hstr += f' {lat_yc} {lon_xc} {depth/1000.0} {mt.magnitude} 0 srcid_{eid}'
'''

idx = pd.IndexSlice
solu_df = recip_record.solutions_df
stat_df = recip_record.stations_df
l_recip_cmtsolutions = []
l_recip_cmtstations  = []
proj_id = recip_record.proj_id
for eidx, edf in rgf_cmt_data_df.groupby(level='eid'):
    eid    = eidx
    print(f'eid: {eid}')
    mt     = d_fwd_src[eid]
    for tidx, tdf in edf.groupby(level='trid'):
        date      = datetime.datetime.now()
        lon_xc    = solu_df.loc[(eidx,0),'lon_xc']
        lat_yc    = solu_df.loc[(eidx,0),'lat_yc']
        depth     = solu_df.loc[(eidx,0),'depth']
        elevation = 0.
        network   = stat_df.loc[(tidx,0,eidx,0),'network']
        stat_header = stat_h(name=f'Reciprocal-Station:{tidx}',
                             lat_yc=lat_yc,
                             lon_xc=lon_xc,
                             depth=depth,
                             elevation=elevation,
                             network=network,
                             proj_id=proj_id,
                             eid=eid,
                             sid=eid,
                             trid=tidx,
                             gid=0)
        l_recip_cmtstations.append(stat_header)
        cmt_lon_xc    = stat_df.loc[(tidx,0,eidx,0),'lon_xc']
        cmt_lat_yc    = stat_df.loc[(tidx,0,eidx,0),'lat_yc']
        cmt_depth     = stat_df.loc[(tidx,0,eidx,0),'depth']
            
    cmt_header = cmt_h(ename=f'Reciprocal-CMT:{eid}',
                       lat_yc=cmt_lat_yc,
                       lon_xc=cmt_lon_xc,
                       depth=cmt_depth,
                       tshift=0,
                       date=date,
                       hdur=0,
                       mt=mt,
                       proj_id=proj_id,
                       eid=eid,
                       sid=eid)

    l_recip_cmtsolutions.append(cmt_header)
    
constructed_record = RecordHeader(name=f'Reciprocal of:{recip_record.name}',
                                  solutions_h=l_recip_cmtsolutions,
                                  stations_h=l_recip_cmtstations,
                                  proj_id=proj_id,
                                  rid=recip_record.rid,
                                  iter_id=recip_record.iter_id,
                                  is_reciprocal=False)

#sid0_df['eid','name'].apply((lambda x: x+1)())

In [ ]:
constructed_record